In [23]:
# import library
import pandas as pd
import numpy as np
from bokeh.io import output_notebook
from bokeh.io import show, output_file
from bokeh.models import HoverTool
from bokeh.plotting import figure
from bokeh.models import FactorRange
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
%matplotlib inline
output_notebook()

Loading BokehJS ...

In [2]:
# set variables

In [3]:
# define functions
def zipcodeMapper(code):
    code = str(int(code)).zfill(5)
    myzip = zipcode.isequal(code)
    try:
        return myzip.state
    except:
        return 'error'

In [4]:
# calculate variables

In [5]:
# readin data

zip_df  = pd.read_csv('free-zipcode-database.csv')
zip_df = zip_df[['Zipcode','State']].drop_duplicates()
zip_df['Zip'] = zip_df['Zipcode'].apply(lambda x:str(int(x)).zfill(5))
zip_df = zip_df.drop('Zipcode',axis=1)

dat = pd.read_csv('re_data.csv',index_col=0)

/Users/pengdong/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# EDA334
# check null values
print(len(dat))
dat.isnull().sum()
dat = dat.dropna()
print(dat.dtypes)

967


Maturity Date               15
Loan Amount                  0
Zip                         12
Property Value               0
Year Built                  23
Net Operating Income         0
Effective Gross Income       0
Total Operating Expenses     0
Maintenance Expense          0
Parking Expense              0
Taxes Expense                0
Insurance Expense            0
Utilities Expense            0
Payroll Expense              0
dtype: int64

Maturity Date                object
Loan Amount                 float64
Zip                         float64
Property Value               object
Year Built                   object
Net Operating Income        float64
Effective Gross Income      float64
Total Operating Expenses    float64
Maintenance Expense         float64
Parking Expense             float64
Taxes Expense               float64
Insurance Expense           float64
Utilities Expense           float64
Payroll Expense             float64
dtype: object


In [7]:
dat['Zip'] = dat['Zip'].apply(lambda x:str(int(x)).zfill(5))
dat = pd.merge(dat,zip_df,on = 'Zip',how = 'left')
dat['Property Value'] = dat['Property Value'].apply(float)
# what the fk

ValueError: could not convert string to float: 'Error'

In [8]:
dat = dat[dat['Property Value'] != 'Error']
dat['Property Value'] = dat['Property Value'].apply(float)

In [9]:
dat.dtypes

Maturity Date                object
Loan Amount                 float64
Zip                          object
Property Value              float64
Year Built                   object
Net Operating Income        float64
Effective Gross Income      float64
Total Operating Expenses    float64
Maintenance Expense         float64
Parking Expense             float64
Taxes Expense               float64
Insurance Expense           float64
Utilities Expense           float64
Payroll Expense             float64
State                        object
dtype: object

In [10]:
# 2 - a
dat_2a = dat.groupby('State',as_index=False).agg({'Loan Amount':'mean'}).sort_values(by = 'Loan Amount',ascending = False)
print('the state with highest average loan amount is {}'.format(dat_2a.iloc[0,0]))

the state with highest average loan amount is OH


In [11]:
# 2 - b
dat_2b = dat.groupby('State',as_index=False).agg({'Taxes Expense':'sum','Property Value':'sum'})                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

dat_2b['ratio'] = dat_2b['Taxes Expense']/dat_2b['Property Value']
dat_2b = dat_2b.sort_values(by = 'ratio',ascending = False)
print('the state with highest taxes as a % of property value is {}'.format(dat_2b.iloc[0,0]))

the state with highest taxes as a % of property value is RI


In [12]:
# 2 - c
dat_2c_origin = dat.copy()
dat_2c_origin['ratio'] = dat_2c_origin['Maintenance Expense']/dat_2c_origin['Property Value']
dat_2c = dat_2c_origin.corr()['ratio'].apply(abs).sort_values()# only focus on abs of corr
print("""strongest predictor might be {}, but this is just a rough guess, 
because in this way, we can only identify linear relations between features""".format(dat_2c.index[-2]))

strongest predictor might be Maintenance Expense, but this is just a rough guess, 
because in this way, we can only identify linear relations between features


In [13]:
# 2 - d
dat_2d_origin = dat.copy()
dat_2d_origin = dat_2d_origin[dat_2d_origin['Property Value'] != 0]
dat_2d_origin['ratio'] = dat_2d_origin['Loan Amount']/dat_2d_origin['Property Value']

dat_2d = dat_2d_origin.ratio.describe()
print("""
median:{}
max:{}
min:{}
variance:{}
""".format(dat_2d['50%'],dat_2d['max'],dat_2d['min'],dat_2d['std']**2))


median:0.6637972465882658
max:1.6483758018592456
min:0.0
variance:0.06959396062440731



In [28]:
# 2 - e
dat_2e = dat.copy()
expense_col = [x for x in dat_2e.columns if 'Expense' in x and 'Expenses' not in x]
dat_2e['Total Expenses'] = dat_2e['Total Operating Expenses']
for col in expense_col:
    dat_2e['Total Expenses'] = dat_2e['Total Expenses']+dat_2e[col]
dat_2e['Expenses per Unit'] = dat_2e['Total Expenses']/dat_2e['Property Value']

In [35]:
print('the correlation between Property Value and Expenses per Unit is {} which means there is no strong correlation'\
      .format(dat_2e[['Property Value','Expenses per Unit']].corr().iloc[0,1]))

the correlation between Property Value and Expenses per Unit is 0.0642413669131821 which means there is no strong correlation


In [30]:
# 3 - a
dat_3_a = dat.groupby('State',as_index=False).agg({'Total Operating Expenses':'sum','Effective Gross Income':'sum'})
dat_3_a['Expense ratio'] = dat_3_a['Total Operating Expenses']/dat_3_a['Effective Gross Income']

dat_3_a = dat_3_a[['State','Expense ratio']].sort_values(by = 'Expense ratio',ascending = False)

x3a = list(dat_3_a.State)
y3a = list(dat_3_a['Expense ratio'])
x = np.arange(len(x3a))
p = figure(x_range = x3a,plot_height=250,plot_width = 750,title = 'Expense Ratio Bar Chart')
p.vbar(x = x3a,top = y3a,width = 0.7)
p.yaxis.axis_label = 'Expense Ratio'
p.xaxis.axis_label = 'State'
p.xaxis.major_label_orientation = 1
show(p)

GlyphRenderer(id='815938cf-1e28-4c39-ad08-081c078fb022', ...)

In [31]:
# 3 - b
def errorSignal(x):
    try:
        tmp = int(x)
        return 'healthy'
    except:
        return 'error'
dat['built year error signal'] = dat['Year Built'].apply(errorSignal)
dat = dat[dat['built year error signal'] == 'healthy']
dat['Property Age'] = 2017 - dat['Year Built'].apply(int)

x = dat['Property Age']
y = dat['Total Operating Expenses']
p = figure(plot_height = 500,plot_width = 500,title = 'Relation Between Property Age and Expenses')
p.scatter(x,y)
p.yaxis.axis_label = 'Total Operating Expenses'
p.xaxis.axis_label = 'Property Age'
show(p)

GlyphRenderer(id='c4549120-5a98-408d-8ede-7901f990bb66', ...)

In [32]:
dat['Maturity Date'] = dat['Maturity Date'] \
    .apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
dat['Fiscal Year'] = dat['Maturity Date'].dt.year
dat['Fiscal Quarter'] = dat['Maturity Date'].dt.quarter

In [33]:
def appendFiscalQuarter(row):
    return str(row[0])+'Q'+str(row[1])
dat['Fiscal Year Quarter'] = dat[['Fiscal Year','Fiscal Quarter']].apply(appendFiscalQuarter,axis=1)

In [44]:
dat_3_c = dat.groupby('Fiscal Year Quarter',as_index=False).agg({'Loan Amount':'sum'}).sort_values(by = 'Fiscal Year Quarter' )
x3c = dat_3_c['Fiscal Year Quarter']
y3c = dat_3_c['Loan Amount']
p = figure(x_range = FactorRange(*x3c),plot_height =250,plot_width = 1000,title = 'Stacked Loan Amount by Fiscal Quarter')
p.line(x = x3c,y = y3c)
p.xaxis.major_label_orientation = 1
show(p)

GlyphRenderer(id='55a119fb-e583-4897-a3d1-b53f526303e8', ...)

In [78]:
# 4
dat_4_origin = dat.copy()
dat_4_origin = dat_4_origin[dat_4_origin['Property Value'] != 0]
dat_4_origin['ratio'] = dat_4_origin['Loan Amount']/dat_4_origin['Property Value']
dat_4_origin['Property Value Inverse'] = 1/dat_4_origin['Property Value']
dat_4_origin['Days to Maturity'] = (dat_4_origin['Maturity Date'].dt.date - datetime.today().date()).dt.days
dat_4 = dat_4_origin.drop(['Maturity Date',
                           'Loan Amount',
                           'Year Built',
                           'Zip','built year error signal','Fiscal Year', 'Fiscal Quarter', 'Fiscal Year Quarter'],axis = 1)


dat_4_dummies = pd.get_dummies(dat_4)

signal = np.random.uniform(size = len(dat_4_dummies))
dat_4_train = dat_4_dummies[signal<=0.7]
dat_4_test = dat_4_dummies[signal>0.7]

x_train = dat_4_train.drop('ratio',axis = 1)
x_test = dat_4_test.drop('ratio',axis = 1)
y_train = dat_4_train.ratio
y_test = dat_4_test.ratio

rf = RandomForestRegressor(n_estimators=500,n_jobs= 4)

param = {'max_depth':list(range(5,16))+[None]}
model = GridSearchCV(rf,param)
model.fit(x_train,y_train)
prediction_train = model.predict(x_train)
prediction_test = model.predict(x_test)
print("""
In Sample:
R2:{}
MSE:{}
Out of Sample:
R2:{}
MSE:{}
""".format(r2_score(y_train,prediction_train),
           mean_squared_error(y_train,prediction_train),
           r2_score(y_test,prediction_test),
           mean_squared_error(y_test,prediction_test)))

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


In Sample:
R2:0.46169847659372976
MSE:0.03706599077045559
Out of Sample:
R2:0.2636431039750824
MSE:0.05222322897562938

